In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: PhasorAUD – A looping interactive Audification of 1D data

This notebook introduces and demonstrates usage of the PhasorAUD sonecule.
- The sonecule enables a simple Audification of a 1-dimensional data series.
- It can be initialized with 
  - a pya Asig (i.e. audio signal using pya)
    - `PhasorAUD(asig, sr=None, channels=None)`
  - a pandas DataFrame or Series
    - `PhasorAUD.from_df(df, sr=44100, time_column=None, columns=None)`
  - a numpy ndarray
    - `PhasorAUD.from_np(data, sr=44100, time_column=None, columns=None)`
- Preprocessing such as time stretching, slicing, filtering is offered by specialized functions, either in pya (iirfilter, stretch) or libraries such as scipy.signal - correspondingly processed signals can be passed into Audification modules for audition and interaction.
- The current PhasorAUD uses SuperCollider3, controlled via sc3nb, as Backend and therein uses Phasor and BufRd UGens for audifications to scan the buffer around a chosen position `relstart`, which allows looped playback of a user-selectable data segment in the data buffer.
- The synth is mutable, i.e., its parameters can be controlled interactively (code or GUI)
  - to enable pause/resume
  - with rate control (note: not band-limited!) via parameter `rate`
  - with amplitude control via parameter `amp`
  - with stereo position control via parameter `pan`
  - with relative start and end point control (in %), via parameter `relstart` and `relend`
  - with reset position control via parameter `respos` to which synth jumps on a trigger
  - with a trigger to immediately go to `respos` 
  - with onset (in schedule)

Let's get started. First some imports and settings and startup of sonecules

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

In [ ]:
df = dataframes['eeg'].loc[:, [1,4,7,10]]
df.plot(subplots=True);
# df.columns

## Usage Demo for the PhasorAUD Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.bufferson import PhasorAUD

We use the realtime mode of the context here.

In [ ]:
ctx.enable_realtime();

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

Note that PhasorAUD always loops between `relstart` and `relend`

In [ ]:
# create the sonecule from data (e.g. channel 7 of the EEG data set )
aud1 = PhasorAUD.from_df(df, sr=256, columns=7)

# reset the timeline 
ctx.timeline.reset()

# schedule the event (which is just one: to start the synth)
aud1.schedule(at=0, rate=5, pan=0, amp=0.5).start()

# plot the data (just for fun)
df.plot(subplots=True);

to stop the audification use

In [ ]:
aud1.stop()

In [ ]:
# here a very slow and low-frequency audification
aud1.reschedule(at=0, rate=2, pan=0, relstart=0.5, relend=0.8, amp=1).start()

Now let's change the rate to 50 for faster temporal compression

In [ ]:
aud1.set(rate=50, amp=0.2, pan=0)

and set the interval between 20% and 30% of the buffer

In [ ]:
aud1.set(relstart=0.20, relend=0.30)

In [ ]:
aud1.stop()

PhasorAUD offers to set the relative start and end position `relstart and relend` in which the phasor cycles.
- i.e. once relend is reached it jumps to relstart

In [ ]:
# to start the sonification at a certain position, without looping
aud1.reschedule(at=0, rate=40, pan=1, relstart=0.3, amp=0.8).start()

The following shows howto parameterize differently using a helper variable `pos` for the start position and `wid` for the width in percent. 

In [ ]:
pos = 0
wid = 0.1
aud1.set(relstart=pos, relend=pos+wid)

In [ ]:
# run this cell some dozens of times to step-by-step move forward by 1%
pos += 0.01; aud1.set(relstart=pos, relend=pos+wid)

you will hear how the timbre and pitch change over the data.

In [ ]:
aud1.stop()

Now we can easily control the Audification with some sliders
* move the startpos slider to skim through the audification
* control rate and trigger rate independently

In [ ]:
from ipywidgets import interactive
aud1.reschedule(at=0, rate=50, pan=0, relstart=0.0, amp=0.8).start() 
def aud_gui(relstart=0, wid=0.1, rate=50):
    aud1.set(relstart=relstart, relend=relstart+wid, rate=rate)
interactive(aud_gui, relstart=(0, 1, 0.005), wid=(0,0.5,0.005), rate=(1, 200, 1)) 

In [ ]:
# and stop when done
aud1.stop()

In [ ]:
ctx.stop()

**Signal Conditioning**

* BasicAUD and PhasorAUD don't offer filtering or distortion, or multi-channel capabilities.
* These, however, are made available in more specialized Sonecules of the AUD family.
* However, some signal conditionings are better applied before audition anyway!
* Modifications such as applying a time scale modification (aka time stretching, i.e. rescaling the time without modifying the spectrum), is for instance well done in pya using Asig.stretch(factor) as shown here for a selected channel and time interval and stretch factor in a one-liner
  
        aud1 = PhasorAUD(my_asig[{1.5:5.2},['channelname']].stretch(3.5))
* so while Sonecules probably don't do it all, combinations with pandas and pya functions enable swift, and flexible implementations of what is needed.

Here howto perform a time stretching prior to audification using pya. Note that the stretch method is not yet in the official branch - monkey patching is done in the prepare_data.ipynb used above

In [ ]:
from pya import startup
s = startup()

The following line shows howto directly playback data (aka audification), demonstrating the daisy chaining logic of pya: the data is
- normalized to 0.5
- converted to stereo
- played at a 30x speed up (immediately since onset defaults to 0)
- then time stretched to 300%
- and played again with same 30x speed up, but starting 2 seconds later (onset=2)
This all is done in a single readable line of code.

In [ ]:
Asig(df.loc[:, 7].values, sr=256).norm(0.5).stereo().play(rate=30).stretch(3).play(onset=2, rate=30)

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# create or load your data, e.g.

# chaotic series from the logistic equation
a, x, xs = 3.5, 0.2, []
for i in range(30000): 
    x = a * x * (1-x); 
    a += 0.000016
    xs.append(x)
data = np.array(xs)

# or load data
# data = pd.read_csv("your_csv_file.csv", delimiter=",")
# data = pd.read_excel("your_excel_file.xlsc") # see pandas documenation
a1 = Asig(data, sr=10000)
plt.figure();a1.plot(lw=0.01)
a1

In [ ]:
# load your data / select your data
myasig = a1

# sonecule for your synth with defaults and bounds
aud1 = PhasorAUD(a1)

# reset the timeline
ctx.timeline.reset() 

# finally start the realtime playback at a given rate
aud1.schedule(at=0, rate=0.2, amp=0.2).start()

# here some scheduled interactions
for t in np.linspace(3,8,40):
    with ctx.at(t):
        pos = 0.5 + 0.5 * np.sin(2 * np.pi*t*1)
        aud1.set(relstart=pos, relend=pos+0.01)
with ctx.at(8):
    aud1.stop()

# if needed: plot the timeline using 
ctx.timeline.plot()

In [ ]:
ctx.stop()

In [ ]:
ctx.close()  # close the mesonic context, exits backend gracefully